In [1]:
import pandas as pd
import numpy as np

print("-------------------------------------------------------")
print("LOADING DATA")
print("-------------------------------------------------------")

df = pd.read_csv("data.csv")
print("Loaded data shape:", df.shape)
print(df.head(), "\n")



-------------------------------------------------------
LOADING DATA
-------------------------------------------------------
Loaded data shape: (320853, 14)
   Year      Country  CountryCode        Variable  VariableCode  \
0  2007  Afghanistan            4  Establishments             1   
1  2008  Afghanistan            4  Establishments             1   
2  2009  Afghanistan            4  Establishments             1   
3  2010  Afghanistan            4  Establishments             1   
4  2011  Afghanistan            4  Establishments             1   

   UnconsolidatedVariableCode UnconsolidatedVariableName ActivityCode  \
0                           1   Number of establishments            C   
1                           1   Number of establishments            C   
2                           1   Number of establishments            C   
3                           1   Number of establishments            C   
4                           1   Number of establishments            C   

 

In [2]:
print("-------------------------------------------------------")
print("STEP 1: BASIC CLEANING")
print("-------------------------------------------------------")

# Remove empty essential fields
df = df.dropna(subset=["Year", "CountryCode", "VariableCode", "ActivityCode"])
df = df.reset_index(drop=True)

# Convert numeric fields properly
df["Year"] = df["Year"].astype(int)
df["VariableCode"] = df["VariableCode"].astype(int)

# ActivityCode can be letters (C, D, E), so convert to string
df["ActivityCode"] = df["ActivityCode"].astype(str).str.strip()

# Convert ValueUSD to numeric
df["ValueUSD"] = pd.to_numeric(df["ValueUSD"], errors="coerce")

print("Step 1 completed. Shape:", df.shape, "\n")

-------------------------------------------------------
STEP 1: BASIC CLEANING
-------------------------------------------------------
Step 1 completed. Shape: (320853, 14) 



In [3]:
print("-------------------------------------------------------")
print("STEP 2: AVAILABLE VARIABLE CODES")
print("-------------------------------------------------------")

available_varcodes = (
    df["VariableCode"]
    .dropna()
    .unique()
    .astype(int)
    .tolist()
)

print("Variable codes found:", available_varcodes)
print(df[["VariableCode", "Variable"]].drop_duplicates().head(20), "\n")


-------------------------------------------------------
STEP 2: AVAILABLE VARIABLE CODES
-------------------------------------------------------
Variable codes found: [1, 4, 5, 14, 31, 20, 21]
      VariableCode                       Variable
0                1                 Establishments
155              4                      Employees
298              5             Wages and salaries
435             14                         Output
645             31               Female employees
2512            20                    Value added
2959            21  Gross fixed capital formation 



In [4]:
print("-------------------------------------------------------")
print("STEP 3: DEFINE VARIABLE GROUPS")
print("-------------------------------------------------------")

VA_CODES = [20]
GFCF_CODES = [21]
EMP_CODES = [4]
ESTAB_CODES = [1]

df["MetricGroup"] = np.where(df["VariableCode"].isin(VA_CODES), "ValueAdded",
                        np.where(df["VariableCode"].isin(GFCF_CODES), "CapitalFormation",
                        np.where(df["VariableCode"].isin(EMP_CODES), "Employment",
                        np.where(df["VariableCode"].isin(ESTAB_CODES), "Establishments",
                        "Other"))))

print(df[["VariableCode", "Variable", "MetricGroup"]].head(20), "\n")

-------------------------------------------------------
STEP 3: DEFINE VARIABLE GROUPS
-------------------------------------------------------
    VariableCode        Variable     MetricGroup
0              1  Establishments  Establishments
1              1  Establishments  Establishments
2              1  Establishments  Establishments
3              1  Establishments  Establishments
4              1  Establishments  Establishments
5              1  Establishments  Establishments
6              1  Establishments  Establishments
7              1  Establishments  Establishments
8              1  Establishments  Establishments
9              1  Establishments  Establishments
10             1  Establishments  Establishments
11             1  Establishments  Establishments
12             1  Establishments  Establishments
13             1  Establishments  Establishments
14             1  Establishments  Establishments
15             1  Establishments  Establishments
16             1  Establ

In [5]:
# ============================================================
# STEP 4 — FILTER
# ============================================================

print("-------------------------------------------------------")
print("STEP 4: FILTER USEFUL VARIABLES")
print("-------------------------------------------------------")

df_filtered = df[df["MetricGroup"] != "Other"].copy()
print("Filtered shape:", df_filtered.shape, "\n")

-------------------------------------------------------
STEP 4: FILTER USEFUL VARIABLES
-------------------------------------------------------
Filtered shape: (192989, 15) 



In [6]:
# ============================================================
# STEP 5 — FEATURE CODE
# ============================================================

print("-------------------------------------------------------")
print("STEP 5: CREATE FEATURE CODES")
print("-------------------------------------------------------")

df_filtered["FeatureCode"] = (
    df_filtered["MetricGroup"] + "_" + df_filtered["VariableCode"].astype(str)
)

print(df_filtered[["VariableCode", "MetricGroup", "FeatureCode"]].head(20), "\n")


-------------------------------------------------------
STEP 5: CREATE FEATURE CODES
-------------------------------------------------------
    VariableCode     MetricGroup       FeatureCode
0              1  Establishments  Establishments_1
1              1  Establishments  Establishments_1
2              1  Establishments  Establishments_1
3              1  Establishments  Establishments_1
4              1  Establishments  Establishments_1
5              1  Establishments  Establishments_1
6              1  Establishments  Establishments_1
7              1  Establishments  Establishments_1
8              1  Establishments  Establishments_1
9              1  Establishments  Establishments_1
10             1  Establishments  Establishments_1
11             1  Establishments  Establishments_1
12             1  Establishments  Establishments_1
13             1  Establishments  Establishments_1
14             1  Establishments  Establishments_1
15             1  Establishments  Establish

In [7]:
# ============================================================
# STEP 6 — WIDE FORMAT
# ============================================================

print("-------------------------------------------------------")
print("STEP 6: PIVOT TO WIDE FORMAT")
print("-------------------------------------------------------")

df_wide = df_filtered.pivot_table(
    index=["CountryCode", "Country", "Year", "ActivityCode"],
    columns="FeatureCode",
    values="ValueUSD",
    aggfunc="mean"
)

df_wide = df_wide.reset_index()

print("Wide shape:", df_wide.shape)
print(df_wide.head(), "\n")


-------------------------------------------------------
STEP 6: PIVOT TO WIDE FORMAT
-------------------------------------------------------
Wide shape: (63213, 8)
FeatureCode  CountryCode      Country  Year ActivityCode  CapitalFormation_21  \
0                      4  Afghanistan  2001            D                  NaN   
1                      4  Afghanistan  2002           15                  NaN   
2                      4  Afghanistan  2002           17                  NaN   
3                      4  Afghanistan  2002           20                  NaN   
4                      4  Afghanistan  2002           22                  NaN   

FeatureCode  Employment_4  Establishments_1  ValueAdded_20  
0                   739.0             153.0            NaN  
1                  1398.0              57.0            NaN  
2                   830.0              11.0            NaN  
3                   723.0              14.0            NaN  
4                   680.0              33.0 

In [8]:
# ============================================================
# STEP 7 — IMPUTE
# ============================================================

print("-------------------------------------------------------")
print("STEP 7: IMPUTATION")
print("-------------------------------------------------------")

for col in df_wide.columns:
    if col not in ["CountryCode", "Country", "Year", "ActivityCode"]:
        df_wide[col] = df_wide[col].fillna(df_wide[col].median())

print("Imputation complete.\n")

-------------------------------------------------------
STEP 7: IMPUTATION
-------------------------------------------------------
Imputation complete.



In [9]:
# ============================================================
# STEP 8 — FINAL SORT
# ============================================================

print("-------------------------------------------------------")
print("STEP 8: FINAL SORT")
print("-------------------------------------------------------")

df_wide = df_wide.sort_values(["CountryCode", "ActivityCode", "Year"])
df_wide = df_wide.reset_index(drop=True)

print("Final shape:", df_wide.shape, "\n")

-------------------------------------------------------
STEP 8: FINAL SORT
-------------------------------------------------------
Final shape: (63213, 8) 



In [10]:
# ============================================================
# STEP 9 — SAVE OUTPUT
# ============================================================

print("-------------------------------------------------------")
print("STEP 9: SAVE FILES")
print("-------------------------------------------------------")

df_wide.to_csv("industry_clean.csv", index=False)
df_wide.to_parquet("industry_clean.parquet", index=False)

print("Saved:")
print("✔ industry_clean.csv")
print("✔ industry_clean.parquet")
print("PROCESS COMPLETED SUCCESSFULLY 🎉")
print("-------------------------------------------------------")

-------------------------------------------------------
STEP 9: SAVE FILES
-------------------------------------------------------
Saved:
✔ industry_clean.csv
✔ industry_clean.parquet
PROCESS COMPLETED SUCCESSFULLY 🎉
-------------------------------------------------------
